# Neural Net: Lunch and Learn

Seth Weidman    

06/15/2017

# What we'll cover

## What we'll cover

* Reminder of conceptually, what neural nets are.

* Show that even with this simple understanding, we can start to use them to solve problems.

* Show how to build deeper nets from scratch, using only built-in Python functionality and `numpy`.

# Neural Nets: What is Going On?

<img src='img/neural_net_wtf.png' height=200>

What does this actually mean?

## Neural nets: what is going on?

* Neural nets are nested functions, where the functions applied to the inputs alternate between two types:
    * Linear functions, represented by matrix multiplications
    * Non linear "activation" functions
* Because they are functions, we can represent the neural net making a prediction mathematically. For example, if $X$ is the input, then the prediction $P$ is just: 

$$ P = D(C(B(A(X, V)), W)) $$

where $A$, $B$, $C$, and $D$ are functions and $V$ and $W$ are weight matrices.

## Review from last time (Pt. 2)

* This prediction is compared to the actual value we were trying to predict, $Y$, and a loss is computed, for example:

$$ L = (Y - P) ^ 2 $$

* And then the weights are adjusted so that this loss will be reduced:

$$ W = W - \frac{\partial L}{\partial W}$$

$$ V = V - \frac{\partial L}{\partial V}$$

## Key point

* This process - of computing derivatives to continually update the weights in a neural net - works because of **the chain rule from calculus**.

## We can do basic neural nets!

<img src="img/neural_net_check.png">

We got this.

# Delving into the math

## Delving into the math

Recall: each "step" is just a function applied to some input that results in some output.

Going deeper, each individual weight makes a contribution to the loss in some way.

So, say we have a neural net with just one hidden layer. If the loss of a neural net on a given observation $ X $ is: 

$$ L = L(D(C(B(A(X, V)), W))) $$

we can use the chain rule the explicitly compute the loss with respect to each of the individual weights:

## Delving into the math

$$ \frac{\partial L}{\partial W} = \frac{\partial C}{\partial W} * \frac{\partial P}{\partial C} * \frac{\partial L}{\partial P} $$

_Note:_ These are matrix multiplications.

$$ \frac{\partial L}{\partial V} = \frac{\partial A}{\partial V} * \frac{\partial B}{\partial A} * \frac{\partial C}{\partial B} * \frac{\partial P}{\partial C} * \frac{\partial L}{\partial P} $$

_Note:_ To get all the weight updates to line up correctly, some of these are elementwise multiplications and some of these are matrix multiplications.

**Key point:** Each one of these individual computations is actually quite simple. I cover what each one of these is in a blog post [here](http://sethweidman.com/neural_net_post_2) and a SlideShare presentation [here](https://www.slideshare.net/SethHWeidman/neural-nets-from-scratch-72835271).

# Using this to learn MNIST

##  Using this to learn MNIST

**Imports**

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import fetch_mldata
import numpy as np
from sklearn.model_selection import train_test_split

##  Using this to learn MNIST

**Read in the MNIST Data**

In [ ]:
mnist = fetch_mldata('MNIST original') 

In [ ]:
def get_mnist_X_Y(mnist):
    data = mnist.data
    X = (data - data.min()) * 1.0 / (data.max() - data.min())
    target = mnist.target
    Y = np.zeros((len(target), 10))
    for i in range(len(target)):
        Y[i][int(target[i])] = 1 
    print("Number of images: ", X.shape[0])
    return X, Y

In [ ]:
X, Y = get_mnist_X_Y(mnist)

In [ ]:
train_prop = 0.9
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, 
    test_size=1-train_prop, 
    random_state=1)

##  Using this to learn MNIST

**Visualize the images**

In [ ]:
def visualize_image(index):
    target = mnist.target
    print("Label: ", int(target[index]))
    plt.imshow(1.0 - X[index].reshape(28,28), cmap='gray')
    plt.show() 

In [ ]:
visualize_image(0)

##  Using this to learn MNIST

**Train the neural net for one epoch**

In [ ]:
def _sigmoid(x):
    return 1.0/(1.0+np.exp(-x))

In [ ]:
def learn(X, Y, num_iter):
    np.random.seed(2)
    V = np.random.randn(784, 50)
    W = np.random.randn(50, 10)
    for j in range(num_iter):
        i = np.random.randint(0, num_iter)
        x = np.array(X[i], ndmin=2)
        y = np.array(Y[i], ndmin=2)
        A = np.dot(x,V)
        B = _sigmoid(A)
        C = np.dot(B,W)
        P = _sigmoid(C)
        sum_P = np.sum(P)
        L = 0.5 * (y - P) ** 2
        dLdP = -1.0 * (y-P)
        dPdC = _sigmoid(C) * (1-_sigmoid(C))
        dLdC = dLdP * dPdC
        dCdW = B.T
        dLdW = np.dot(dCdW, dLdC)
        dCdB = W.T
        dLdB = np.dot(dLdC, dCdB)
        dBdA = _sigmoid(A) * (1-_sigmoid(A))
        dLdA = dLdB * dBdA
        dAdV = x.T
        dLdV = np.dot(dAdV, dLdA)
        W -= dLdW
        V -= dLdV
    return V, W  

In [ ]:
  def predict(X, V, W):
    A = np.dot(X,V)
    B = _sigmoid(A)
    C = np.dot(B,W)
    P = _sigmoid(C)
    return P

In [ ]:
V, W = learn(X_train, Y_train, num_iter=X_train.shape[0])

In [ ]:
P = predict(X_test, V, W)

In [ ]:
preds = [np.argmax(x) for x in P]
actuals = [np.argmax(x) for x in Y_test]

accuracy = sum(np.array(preds) == np.array(actuals)) * 1.0 / len(preds)
print("Neural Net MNIST Classification Accuracy:", round(accuracy, 3) * 100, "percent")

91.3% accuracy. Not bad for a naive approach with zero tricks. However...

...it's far from optimal: [see here](http://rodrigob.github.io/are_we_there_yet/build/classification_datasets_results.html)

<img src='img/MNIST_performance.png' height=200>

# How can we do better?

## How can we do better?

* Building deeper nets
* Changing "learning rates" / using "learning rate decay"
* Adding "dropout"
* ...and that doesn't even cover many of the cutting edge techniques listed above

And of course, we have to build it from scratch.

**"What I cannot build, I cannot understand."**

--Richard Feynman

## Before:

With a neural net with one hidden layer, one pass through was 25 manually coded steps. 

_This will quickly get unweildy if we add more hidden layers._

## New understanding: "Layers"

To do "Deep Learning" you have to stop thinking of neural nets as a series of functions $ L = L(D(C(B(A(X, V)), W)) $.

### New understanding: "Layers"

Instead, we'll think of them as a series of layers:

Slide from March talk:

<img src='img/neural_net_layers.png' height=200>

## Coding the new neural network, layers

We'll define a `NeuralNetwork` class that defines a neural network as a series of `Layers`:

In [ ]:
class NeuralNetwork(object):
    def __init__(self, layers, random_seed):
        self.layers = layers
        self.random_seed = random_seed
        for i, layer in enumerate(self.layers):
            setattr(layer, 'random_seed', self.random_seed+i)
            layer.initialize_weights()


    def forwardpass(self, X):
        """ Calculate an output Y for the given input X. """
        X_next = X
        for layer in self.layers:
            X_next = layer.fprop(X_next)
        prediction = X_next
        return prediction

    def loss(self, prediction, Y):
        """ Calculate error on the given data. """
        loss = 0.5 * (Y - prediction) ** 2
        return -1.0 * (Y - prediction)

    def backpropogate(self, loss):
        """ Calculate an output Y for the given input X. """
        loss_next = loss
        for layer in reversed(self.layers):
            loss_next = layer.bprop(loss_next)
        return loss

### Coding the new neural network, layers

In [ ]:
class Layer(object):
    def _setup(self, input_shape):
        """ Setup layer with parameters that are unknown at __init__(). """
        pass

    def fprop(self, input):
        """ Calculate layer output for given input (forward propagation). """
        raise NotImplementedError()

    def bprop(self, output_grad):
        """ Calculate input gradient. """
        raise NotImplementedError()

## Props

<img src="img/andersbll.png">

[Anders' GitHub](https://github.com/andersbll)

### Coding the new neural network, layers

In [ ]:
class Linear(Layer):
    
    random_seed = None
    
    def __init__(self, n_in, n_out, 
                 activation_function):
        self.n_in = n_in
        self.n_out = n_out        
        self.iteration = 0
        self.activation_function = activation_function

    def initialize_weights(self):
        np.random.seed(seed=self.random_seed)
        self.W = np.random.normal(size=(self.n_in, self.n_out))

    def fprop(self, layer_input):
        self.layer_input = layer_input
        self.activation_input = np.dot(layer_input, self.W)
        return self.activation_function(self.activation_input, bprop=False)

    def bprop(self, layer_gradient):
        dOutdActivationInput = self.activation_function(self.activation_input, bprop=True)
        dLayerInputdActivationInput = layer_gradient * dOutdActivationInput
        dActivationOutputdActivationInput = self.layer_input.T
        output_grad = np.dot(dLayerInputdActivationInput, self.W.T)
        weight_update = np.dot(dActivationOutputdActivationInput, dLayerInputdActivationInput)
        W_new = self.W - weight_update
        self.W = W_new
        self.iteration += 1
        return output_grad

### Coding the new neural network, layers - and don't forget about that activation function!

In [ ]:
def sigmoid(x, bprop=False):
    if bprop:
        s = sigmoid(x)
        return s*(1-s)
    else:
        return 1.0/(1.0+np.exp(-x))

## Running this neural network:

In [ ]:
layer1 = Linear(n_in=784, n_out=50, activation_function=sigmoid)
layer2 = Linear(n_in=50, n_out=10, activation_function=sigmoid)

In [ ]:
nn_mnist = NeuralNetwork(
    layers=[layer1, layer2],
    random_seed=2)

In [ ]:
def neural_net_pass(net, x, y):
    pred = net.forwardpass(x)
    loss = net.loss(pred, y)
    net.backpropogate(loss)
    return pred

### Running this neural network

In [ ]:
# Randomly shuffle the indices of the points in the training set:
np.random.seed(4)
train_size = X_train.shape[0]
indices = list(range(train_size))
np.random.shuffle(indices)

In [ ]:
# Loop through every element in the training set: 
for index in indices:
    x = np.array(X_train[index], ndmin=2)
    y = np.array(Y_train[index], ndmin=2)
    neural_net_pass(nn_mnist, x, y)

In [ ]:
P = nn_mnist.forwardpass(X_test)
preds = [np.argmax(x) for x in P]
actuals = [np.argmax(x) for x in Y_test]

accuracy = sum(np.array(preds) == np.array(actuals)) * 1.0 / len(preds)
print("Neural Net MNIST Classification Accuracy:", round(accuracy, 3) * 100, "percent")

A different random weight initialization gave us just 82.3% accuracy this time.

### Running this neural network

#### Turning what we did above into a function

In [ ]:
def train_test_net(net, X_train, Y_train, X_test, Y_test, random_seed):

    # Randomly set the seed
    np.random.seed(random_seed)
    
    # Randomly shuffle the indices of the points in the training set:
    train_size = X_train.shape[0]
    indices = list(range(train_size))
    np.random.shuffle(indices)
    
    # Loop through every element in the training set: 
    for index in indices:
        x = np.array(X_train[index], ndmin=2)
        y = np.array(Y_train[index], ndmin=2)
        neural_net_pass(net, x, y)
        
    P = net.forwardpass(X_test)
    preds = [np.argmax(x) for x in P]
    actuals = [np.argmax(x) for x in Y_test]

    accuracy = sum(np.array(preds) == np.array(actuals)) * 1.0 / len(preds)

    return accuracy

# Deeper networks



Now that we've built a framework allowing us to define networks as a series of layers, we can easily build deeper networks.

In [ ]:
input_layer = Linear(n_in=784, n_out=50, activation_function=sigmoid)
hidden_1 = Linear(n_in=50, n_out=50, activation_function=sigmoid)
hidden_2 = Linear(n_in=50, n_out=50, activation_function=sigmoid)
hidden_3 = Linear(n_in=50, n_out=50, activation_function=sigmoid)
output_layer = Linear(n_in=50, n_out=10, activation_function=sigmoid)

In [ ]:
def print_accuracy_net_layers(layers):
    # Define net
    net = NeuralNetwork(
        layers=layers,
        random_seed=2)

    accuracy_one_hidden = train_test_net(net, X_train, Y_train, X_test, Y_test, 4)
    
    return accuracy_one_hidden

## Deeper networks

In [ ]:
accuracy = print_accuracy_net_layers([input_layer, 
                                      output_layer])
print("The accuracy of a network with one hidden layer was", np.round(accuracy * 100.0, 1))

In [ ]:
accuracy = print_accuracy_net_layers([input_layer, 
                                      hidden_1, 
                                      output_layer])
print("The accuracy of a network with two hidden layers was", np.round(accuracy * 100.0, 1))

In [ ]:
accuracy = print_accuracy_net_layers([input_layer, 
                                      hidden_1, 
                                      hidden_2,
                                      output_layer])
print("The accuracy of a network with three hidden layers was", np.round(accuracy * 100.0, 1))

In [ ]:
accuracy = print_accuracy_net_layers([input_layer, 
                                      hidden_1, 
                                      hidden_2,
                                      hidden_3,
                                      output_layer])
print("The accuracy of a network with four hidden layers was", np.round(accuracy * 100.0, 1))

**"Deep Learning" can help, but we need more "tricks" to really get these things working.**

# Learning rate tuning

<img src="img/bengio.png">

"The learning rate is the single most important hyperparameter and one should always make sure it is tuned."

-[Yoshua Bengio](http://www.iro.umontreal.ca/~bengioy/yoshua_en/)

## Learning rate tuning

In [ ]:
class NeuralNetwork(object):
    def __init__(self, layers, random_seed, learning_rate):
        self.layers = layers
        self.random_seed = random_seed
        self.learning_rate = learning_rate
        for i, layer in enumerate(self.layers):
            setattr(layer, 'random_seed', self.random_seed+i)
            setattr(layer, 'learning_rate', self.learning_rate)
            layer.initialize_weights()


    def forwardpass(self, X):
        """ Calculate an output Y for the given input X. """
        X_next = X
        for layer in self.layers:
            X_next = layer.fprop(X_next)
        prediction = X_next
        return prediction

    def loss(self, prediction, Y):
        """ Calculate error on the given data. """
        loss = 0.5 * (Y - prediction) ** 2
        return -1.0 * (Y - prediction)

    def backpropogate(self, loss):
        """ Calculate an output Y for the given input X. """
        loss_next = loss
        for layer in reversed(self.layers):
            loss_next = layer.bprop(loss_next)
        return loss

## Learning rate tuning

In [ ]:
class Linear(Layer):
    
    random_seed = None
    learning_rate = None
    
    def __init__(self, n_in, n_out, 
                 activation_function):
        self.n_in = n_in
        self.n_out = n_out        
        self.iteration = 0
        self.activation_function = activation_function

    def initialize_weights(self):
        np.random.seed(seed=self.random_seed)
        self.W = np.random.normal(size=(self.n_in, self.n_out))

    def fprop(self, layer_input):
        self.layer_input = layer_input
        self.activation_input = np.dot(layer_input, self.W)
        return self.activation_function(self.activation_input, bprop=False)

    def bprop(self, layer_gradient):
        dOutdActivationInput = self.activation_function(self.activation_input, bprop=True)
        dLayerInputdActivationInput = layer_gradient * dOutdActivationInput
        dActivationOutputdActivationInput = self.layer_input.T
        output_grad = np.dot(dLayerInputdActivationInput, self.W.T)
        weight_update = np.dot(dActivationOutputdActivationInput, dLayerInputdActivationInput)
        W_new = self.W - self.learning_rate * weight_update
        self.W = W_new
        self.iteration += 1
        return output_grad

## Learning rate tuning

In [ ]:
input_layer = Linear(n_in=784, n_out=50, activation_function=sigmoid)
hidden_1 = Linear(n_in=50, n_out=50, activation_function=sigmoid)
output_layer = Linear(n_in=50, n_out=10, activation_function=sigmoid)

In [ ]:
def learning_rate_accuracy(learning_rate):
    
    net = NeuralNetwork(
        layers = [input_layer, hidden_1, output_layer],
        random_seed = 2,
        learning_rate = learning_rate
    )
    
    accuracy_lr = train_test_net(net, X_train, Y_train, X_test, Y_test, 4)
    print("The accuracy of a network with learning rate", learning_rate, "was", np.round(accuracy_lr * 100.0, 1))

    return accuracy_lr

In [ ]:
learning_rates = [0.0001, 0.001, 0.01, 0.1, 1, 10]
lr_accuracies = [learning_rate_accuracy(x) for x in learning_rates]

# Learning rate tuning: getting fancy

Because backpropogation involves multiplying a value by the derivative of the activation function, gradients (that tell the weights how to update) get smaller and smaller as you go get further from the output layer:

<img src="img/sigmoid_deriv_trask.png">
**At most, the gradient can be multiplied by 0.25 at each layer.** More [here](http://iamtrask.github.io/2015/07/12/basic-python-network/)

## Learning rate tuning: getting fancy

In [ ]:
class NeuralNetwork(object):
    def __init__(self, layers, 
                 random_seed, 
                 learning_rate):
        self.layers = layers
        self.random_seed = random_seed
        self.learning_rate = learning_rate
        for i, layer in enumerate(self.layers):
            setattr(layer, 'random_seed', self.random_seed+i)
            setattr(layer, 'learning_rate', 
                    self.learning_rate/(10.0 ** i))
            layer.initialize_weights()

    def forwardpass(self, X):
        """ Calculate an output Y for the given input X. """
        X_next = X
        for layer in self.layers:
            X_next = layer.fprop(X_next)
        prediction = X_next
        return prediction

    def loss(self, prediction, Y):
        """ Calculate error on the given data. """
        loss = 0.5 * (Y - prediction) ** 2
        return -1.0 * (Y - prediction)

    def backpropogate(self, loss):
        """ Calculate an output Y for the given input X. """
        loss_next = loss
        for layer in reversed(self.layers):
            loss_next = layer.bprop(loss_next)
        return loss

## Learning rate tuning: getting fancy

In [ ]:
input_layer = Linear(n_in=784, n_out=50, activation_function=sigmoid)
hidden_1 = Linear(n_in=50, n_out=50, activation_function=sigmoid)
output_layer = Linear(n_in=50, n_out=10, activation_function=sigmoid)

In [ ]:
def learning_rate_accuracy(learning_rate):
    
    net = NeuralNetwork(
        layers = [input_layer, hidden_1, output_layer],
        random_seed = 2,
        learning_rate = learning_rate
    )
    
    accuracy_lr = train_test_net(net, X_train, Y_train, X_test, Y_test, 4)
    
    print("The accuracy of a network with learning rate \n", learning_rate,
          "in the first layer \n", learning_rate / 10.0,
          "in the second layer, and \n", learning_rate / 100.0, 
          "in the third layer is", 
          np.round(accuracy_lr * 100.0, 1), "percent")

    return accuracy_lr

## Learning rate tuning: getting fancy

In [ ]:
learning_rates = [0.01, 0.1, 1, 10]
lr_accuracies = [learning_rate_accuracy(x) for x in learning_rates]

The accuracy is up to 91.8%, up from 90.7% before. We could also view that as roughly a 10% reduction in error rate. 

**Having a higher learning rate in earlier layers vs. later layers does help accuracy.** 

# Learning rate momentum

The weights in a neural net are updated according to:

$$ W =  W - \frac{\partial L}{\partial W}$$

Recall that this is equivalent to doing gradient descent with each parameter:

<img src="img/gradient_descent.png">

This is analogous to a ball rolling down a hill. 

Balls rolling down hills have momentum. So, therefore, should our weights!

## Learning rate momentum

Let's define our weight update $ \frac{\partial L}{\partial W} $ to be $ U_t $. Then, instead of our weight update being $ U_t $ at each time step, it will be:

$$ U_t + \mu * U_{t-1} + \mu^2 * U_{t-2} + ... $$ 

where $\mu$ is a decay parameter between 0 and 1.

This is equivalent to, and often described as, increasing your learning rate when your weight updates are going in the same direction, iteration after iteration, and lowering your learning rate when the opposite is happening.

## Implementing learning rate momentum

In [ ]:
class Linear(Layer):
    
    random_seed = None
    learning_rate = None
    momentum = None
    
    def __init__(self, n_in, n_out, 
                 activation_function):
        self.n_in = n_in
        self.n_out = n_out        
        self.iteration = None
        self.activation_function = activation_function
        self.velocity = None

    def initialize_weights(self):
        np.random.seed(seed=self.random_seed)
        self.W = np.random.normal(size=(self.n_in, self.n_out))
        self.velocity = np.zeros(shape=(self.n_in, self.n_out))

    def fprop(self, layer_input):
        self.layer_input = layer_input
        self.activation_input = np.dot(layer_input, self.W)
        return self.activation_function(self.activation_input, bprop=False)

    def bprop(self, layer_gradient):
        dOutdActivationInput = self.activation_function(self.activation_input, 
                                                        bprop=True)
        dLayerInputdActivationInput = layer_gradient * dOutdActivationInput
        dActivationOutputdActivationInput = self.layer_input.T
        output_grad = np.dot(dLayerInputdActivationInput, self.W.T)
        
        # Update velocity
        weight_update_current = np.dot(dActivationOutputdActivationInput, 
                                       dLayerInputdActivationInput)
        self.velocity = np.add(self.momentum * self.velocity, 
                               self.learning_rate * weight_update_current)
        self.W = self.W - self.velocity
        return output_grad

## Implementing learning rate momentum

In [ ]:
class NeuralNetwork(object):
    def __init__(self, layers, random_seed, learning_rate, momentum):
        self.layers = layers
        self.random_seed = random_seed
        self.momentum = momentum
        self.learning_rate = learning_rate
        for i, layer in enumerate(self.layers):
            setattr(layer, 'random_seed', self.random_seed+i)
            layer.initialize_weights()
            setattr(layer, 'learning_rate', self.learning_rate/(10.0 ** i))
            setattr(layer, 'momentum', self.momentum)

    def forwardpass(self, X):
        """ Calculate an output Y for the given input X. """
        X_next = X
        for layer in self.layers:
            X_next = layer.fprop(X_next)
        prediction = X_next
        return prediction

    def loss(self, prediction, Y):
        """ Calculate error on the given data. """
        loss = 0.5 * (Y - prediction) ** 2
        return -1.0 * (Y - prediction)

    def backpropogate(self, loss):
        """ Calculate an output Y for the given input X. """
        loss_next = loss
        for layer in reversed(self.layers):
            loss_next = layer.bprop(loss_next)
        return loss

## Implementing learning rate momentum

In [ ]:
input_layer = Linear(n_in=784, n_out=50, activation_function=sigmoid)
hidden_1 = Linear(n_in=50, n_out=50, activation_function=sigmoid)
output_layer = Linear(n_in=50, n_out=10, activation_function=sigmoid)

In [ ]:
def learning_rate_accuracy(learning_rate, momentum):
    
    net = NeuralNetwork(
        layers = [input_layer, hidden_1, output_layer],
        random_seed = 2,
        learning_rate = learning_rate,
        momentum = momentum
    )
    
    accuracy_lr = train_test_net(net, X_train, Y_train, X_test, Y_test, 4)
    
    print("The accuracy of a network with learning rate \n", learning_rate,
          "in the first layer \n", learning_rate / 10.0,
          "in the second layer, and \n", learning_rate / 100.0, 
          "in the third layer, and \n momentum", momentum, "is",
          np.round(accuracy_lr * 100.0, 1), "percent")

    return accuracy_lr

## Learning rate momentum results

In [ ]:
learning_rates = [1]
momentum = [0.5, 0.75, 0.9]
for learning_rate in learning_rates:
    for m in momentum:
        learning_rate_accuracy(learning_rate, m)

Here, we get a practical lesson: **throwing the "kitchen sink" of neural net tricks at a problem in unnecessary at best and actually harmful at worst.** 

Indeed, learning rate momentum has been shown to work on Recurrent Neural Nets, not fully connected neural nets as we have here. See 
[Bengio et. al. (2014)](https://arxiv.org/pdf/1212.0901v2.pdf).

# Dropout

Dropout can help prevent neural networks from overfitting. It involves "dropping" a portion of the neurons - that is, setting their values to zero - on each forward pass through the network. 

<img src="img/dropout.png">

This nudges the network toward learning "redundant representations of its data".

## Dropout

In [ ]:
class NeuralNetworkDropout(object):
    def __init__(self, layers, random_seed, 
                 learning_rate, momentum, dropout):
        self.layers = layers
        self.random_seed = random_seed
        self.momentum = momentum
        self.dropout = dropout
        self.learning_rate = learning_rate
        self.iteration = 0
        for i, layer in enumerate(self.layers):
            setattr(layer, 'random_seed', self.random_seed+i)
            layer.initialize_weights()
            setattr(layer, 'learning_rate', self.learning_rate/(10.0 ** i))
            setattr(layer, 'momentum', self.momentum)

    def forwardpass(self, X):
        """ Calculate an output Y for the given input X. """
        X_next = X
        for i, layer in enumerate(self.layers):
            np.random.seed(seed=self.random_seed+i*self.iteration)
            if self.dropout:
                zero_indices = np.random.choice(range(layer.n_in), 
                                                size=int(layer.n_in * (1 - self.dropout)), 
                                                replace=False)
                X_next[:, zero_indices] = 0.0
            X_next = layer.fprop(X_next)
        prediction = X_next
        self.iteration += 1
        return prediction

    def loss(self, prediction, Y):
        """ Calculate error on the given data. """
        loss = 0.5 * (Y - prediction) ** 2
        return -1.0 * (Y - prediction)

    def backpropogate(self, loss):
        """ Calculate an output Y for the given input X. """
        loss_next = loss
        for layer in reversed(self.layers):
            loss_next = layer.bprop(loss_next)
        return loss

## Dropout

In [ ]:
def train_test_net_dropout(net, X_train, Y_train, X_test, Y_test, random_seed):

    # Randomly set the seed
    np.random.seed(random_seed)
    
    # Randomly shuffle the indices of the points in the training set:
    train_size = X_train.shape[0]
    indices = list(range(train_size))
    np.random.shuffle(indices)
    
    # Loop through every element in the training set: 
    for index in indices:
        x = np.array(X_train[index], ndmin=2)
        y = np.array(Y_train[index], ndmin=2)
        neural_net_pass(net, x, y)
        
    net.dropout = None
    P = net.forwardpass(X_test)
    preds = [np.argmax(x) for x in P]
    actuals = [np.argmax(x) for x in Y_test]

    accuracy = sum(np.array(preds) == np.array(actuals)) * 1.0 / len(preds)
    
    return accuracy

## Dropout

In [ ]:
input_layer = Linear(n_in=784, n_out=50, activation_function=sigmoid)
hidden_1 = Linear(n_in=50, n_out=50, activation_function=sigmoid)
output_layer = Linear(n_in=50, n_out=10, activation_function=sigmoid)

In [ ]:
def learning_rate_dropout(dropout):
    
    net = NeuralNetworkDropout(
        layers = [input_layer, hidden_1, output_layer],
        random_seed = 2,
        learning_rate = 1,
        momentum = 0.5,
        dropout = dropout
    )
    
    accuracy_lr = train_test_net_dropout(net, X_train, Y_train, X_test, Y_test, 4)
    
    print("The accuracy of a network with dropout", dropout, "is",
          np.round(accuracy_lr * 100.0, 1), "percent")

    return accuracy_lr

## Testing dropout

In [ ]:
dropouts = [1, 0.5, 0.75, 0.9]
lr_accuracies = [learning_rate_dropout(x) for x in dropouts]

**Adding dropout did _not_ improve accuracy.** Our network was not overfitting in this case.

# Last one, just for fun: DropConnect

As we saw above, the highest performance model on the MNIST data involved "Drop Connect", where a portion of the _weights_ in the neural net are set to zero, as opposed to half the _neurons_.

[Not in TensorFlow!](https://stackoverflow.com/questions/37135885/dropconnect-in-tensorflow)

<img src="img/drop_connect.png">

## DropConnect 

In [ ]:
class NeuralNetwork(object):
    def __init__(self, layers, random_seed, 
                 learning_rate, momentum, drop_connect,
                 dropout=None):
        self.layers = layers
        self.random_seed = random_seed
        self.momentum = momentum
        self.dropout = dropout
        self.drop_connect = drop_connect
        self.learning_rate = learning_rate
        self.iteration = 0
        for i, layer in enumerate(self.layers):
            setattr(layer, 'random_seed', self.random_seed+i)
            layer.initialize_weights()
            setattr(layer, 'learning_rate', self.learning_rate/(10.0 ** i))
            setattr(layer, 'momentum', self.momentum)
            setattr(layer, 'drop_connect', self.drop_connect)

    def forwardpass(self, X):
        """ Calculate an output Y for the given input X. """
        X_next = X
        for i, layer in enumerate(self.layers):
            np.random.seed(seed=self.random_seed+i*self.iteration)
            if self.dropout:
                zero_indices = np.random.choice(range(layer.n_in), 
                                                size=int(layer.n_in * (1 - self.dropout)), 
                                                replace=False)
                X_next[:, zero_indices] = 0.0
            X_next = layer.fprop(X_next)
        prediction = X_next
        self.iteration += 1
        return prediction

    def loss(self, prediction, Y):
        """ Calculate error on the given data. """
        loss = 0.5 * (Y - prediction) ** 2
        return -1.0 * (Y - prediction)

    def backpropogate(self, loss):
        """ Calculate an output Y for the given input X. """
        loss_next = loss
        for layer in reversed(self.layers):
            loss_next = layer.bprop(loss_next)
        return loss

## DropConnect

In [ ]:
def apply_drop_connect_weights(weights, drop_connect):
    new_weights = weights.copy()
    num_weights = new_weights.shape[0] * new_weights.shape[1]
    reshaped_weights = np.reshape(new_weights, (num_weights, 1))
    zero_indices = np.random.choice(range(num_weights), 
                                    size=int(num_weights * (1 - drop_connect)), 
                                    replace=False)
    reshaped_weights[zero_indices, :] = 0.0
    drop_connected_weights = np.reshape(reshaped_weights, new_weights.shape)
    
    return drop_connected_weights

## DropConnect

In [ ]:
class Linear(Layer):
    
    random_seed = None
    learning_rate = None
    momentum = None
    drop_connect = None
    
    def __init__(self, n_in, n_out, 
                 activation_function):
        self.n_in = n_in
        self.n_out = n_out        
        self.iteration = 0
        self.activation_function = activation_function
        self.velocity = None

    def initialize_weights(self):
        np.random.seed(seed=self.random_seed)
        self.W = np.random.normal(size=(self.n_in, self.n_out))
        self.velocity = np.zeros(shape=(self.n_in, self.n_out))

    def fprop(self, layer_input):
        self.layer_input = layer_input
        if self.drop_connect:            
            drop_connected_weights = apply_drop_connect_weights(self.W, 
                                                                self.drop_connect)
            self.activation_input = np.dot(layer_input, 
                                           drop_connected_weights)
        else:
            self.activation_input = np.dot(layer_input, self.W)
        self.iteration += 1
        return self.activation_function(self.activation_input, bprop=False)

    def bprop(self, layer_gradient):
        dOutdActivationInput = self.activation_function(self.activation_input, 
                                                        bprop=True)
        dLayerInputdActivationInput = layer_gradient * dOutdActivationInput
        dActivationOutputdActivationInput = self.layer_input.T
        output_grad = np.dot(dLayerInputdActivationInput, self.W.T)
        
        # Update velocity
        weight_update_current = np.dot(dActivationOutputdActivationInput, 
                                       dLayerInputdActivationInput)
        self.velocity = np.add(self.momentum * self.velocity, 
                               self.learning_rate * weight_update_current)
        self.W = self.W - self.velocity
        return output_grad

## DropConnect

In [ ]:
def train_test_net_drop_connect(net, X_train, Y_train, X_test, Y_test, random_seed):

    # Randomly set the seed
    np.random.seed(random_seed)
    
    # Randomly shuffle the indices of the points in the training set:
    train_size = X_train.shape[0]
    indices = list(range(train_size))
    np.random.shuffle(indices)
    
    # Loop through every element in the training set: 
    for index in indices:
        x = np.array(X_train[index], ndmin=2)
        y = np.array(Y_train[index], ndmin=2)
        neural_net_pass(net, x, y)
        
    net.drop_connect = None
    P = net.forwardpass(X_test)
    preds = [np.argmax(x) for x in P]
    actuals = [np.argmax(x) for x in Y_test]

    accuracy = sum(np.array(preds) == np.array(actuals)) * 1.0 / len(preds)
    
    return accuracy

## DropConnect

In [ ]:
def learning_rate_drop_connect(drop_connect):
    
    net = NeuralNetwork(
        layers = [input_layer, hidden_1, output_layer],
        random_seed = 2,
        learning_rate = 1,
        momentum = 0.5,
        drop_connect = drop_connect
    )

    accuracy_lr = train_test_net_drop_connect(net, X_train, Y_train, X_test, Y_test, 4)
    
    print("The accuracy of a network with drop_connect", 
          drop_connect, "is",
          np.round(accuracy_lr * 100.0, 1), "percent")

    return accuracy_lr

In [ ]:
input_layer = Linear(n_in=784, n_out=50, activation_function=sigmoid)
hidden_1 = Linear(n_in=50, n_out=50, activation_function=sigmoid)
output_layer = Linear(n_in=50, n_out=10, activation_function=sigmoid)

## DropConnect

In [ ]:
drop_connect_values = [1, 0.5, 0.75, 0.9]
lr_accuracies = [learning_rate_drop_connect(x) for x in drop_connect_values]

This network, not being convolutional, is likely _underfitting_ rather than _overfitting_ the data.

# Conclusion

There are a lot of "tricks" to improve the training of neural nets. **Now you know not just conceptually what those tricks are doing, but how to implement them.** You may even be able to implement a few of your own!

## Next time:

We'll examine:
* Different activation functions! (here we only used boring ol' sigmoid)
* Different weight initializations

## Next next time

* Convolutional layers (yes, from scratch)
* Recurrent neural nets (including LSTMs, omg)

## Next steps for you

Go to [the GitHub repo for this talk](https://github.com/SethHWeidman/neural_net_talks).

Star, fork, and contribute!